In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, applications, regularizers

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import math

In [ ]:
train_path = "../input/petfinder-pawpularity-score/train.csv"

train_imgs_path = "../input/petfinder-pawpularity-score/train"

In [ ]:
train_data = pd.read_csv(train_path)
train_data['Id'] = train_data['Id'].map(lambda img_id: img_id+".jpg")
train_data['Pawpularity'] = train_data['Pawpularity']
train_data = train_data[['Id', 'Pawpularity']]

In [ ]:
IMG_SIZE = 244
BATCH_SIZE = 16
EPOCHS = 25
INTIAL_LEARN_RATE = 0.0001

In [ ]:
def give_dataGen(valid_spilt=0.2):
    train_gen = ImageDataGenerator(
                                    rotation_range=30,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    brightness_range=[0.3,0.9],
                                    shear_range=15,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    rescale=1/255,
                                    validation_split=valid_spilt
                                   )

    valid_gen = ImageDataGenerator(rescale=1/255,
                                   validation_split = valid_spilt
                                  )

    train_generator = train_gen.flow_from_dataframe(
                                dataframe=train_data,
                                directory = train_imgs_path,
                                x_col = "Id",
                                y_col = "Pawpularity",
                                target_size = (IMG_SIZE, IMG_SIZE),
                                class_mode = "raw",
                                batch_size = BATCH_SIZE,
                                shuffle = True,
                                subset = "training",
    )

    valid_generator = valid_gen.flow_from_dataframe(
                                dataframe=train_data,
                                directory = train_imgs_path,
                                x_col = "Id",
                                y_col = "Pawpularity",
                                target_size = (IMG_SIZE, IMG_SIZE),
                                class_mode = "raw",
                                batch_size = BATCH_SIZE,
                                shuffle = True,
                                subset = "validation"
    )
    
    return train_generator, valid_generator

In [ ]:
def draw(gen):
    batch = next(gen)
    images = batch[0]
    labels = batch[1]

    plt.figure(figsize=(18,6))
    for i, (img, label) in enumerate(zip(images, labels)):
        plt.imshow(img)
        plt.show()
        if i==15:
            break

In [ ]:
def build_model():
    base = applications.EfficientNetB7(include_top=False, input_shape=[IMG_SIZE,IMG_SIZE,3])
    
    model = tf.keras.Sequential()
    model.add(base)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='relu'))
    
    return model

In [ ]:
build_model().summary()

In [ ]:
def exp_decay(epoch, lr):
    initial_lrate = INTIAL_LEARN_RATE
    k = 0.2
    lrate = initial_lrate * math.exp(-k*epoch)
    print(f"LEARNING RATE : {lrate}")
    return lrate

In [ ]:
callback0 = tf.keras.callbacks.LearningRateScheduler(exp_decay)
for split_percent in [0.3, 0.2, 0.1]:
    train_generator, valid_generator = give_dataGen(valid_spilt=split_percent)
    
    model = model = build_model()
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=INTIAL_LEARN_RATE), loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    callback1 = tf.keras.callbacks.ModelCheckpoint(f"BestModel_{split_percent}_split.h5", 
                                               monitor='val_root_mean_squared_error',save_best_only=True, verbose=1)
    his = model.fit(train_generator, validation_data=valid_generator, epochs=EPOCHS, callbacks=[callback1])
    
    stats = pd.DataFrame(his.history)
    stats.plot()
    plt.show()
    print("\n\n")